EDA

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("/content/180001_game_review.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180001 entries, 0 to 180000
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   review          179950 non-null  object
 1   hours_played    180001 non-null  object
 2   helpful         180001 non-null  object
 3   recommendation  180001 non-null  object
 4   date            180001 non-null  object
 5   game_name       180001 non-null  object
dtypes: object(6)
memory usage: 8.2+ MB


label / binary encoding

In [3]:
df["recommendation"] = df["recommendation"].map({
    1: "Recommended",
    0: "Not Recommended"
})

In [4]:
df

,review,hours_played,helpful,recommendation,date,game_name
0,The game itself is also super fun. The PvP and...,39.9,"1,152",Recommended,14 September,"Warhammer 40,000: Space Marine 2"
1,Never cared much about Warhammer until this ga...,91.5,712,Recommended,13 September,"Warhammer 40,000: Space Marine 2"
2,A salute to all the fallen battle brothers who...,43.3,492,Recommended,14 September,"Warhammer 40,000: Space Marine 2"
3,this game feels like it was made in the mid 20...,16.8,661,Recommended,14 September,"Warhammer 40,000: Space Marine 2"
4,Reminds me of something I've lost. A genuine g...,24.0,557,Recommended,12 September,"Warhammer 40,000: Space Marine 2"
...,...,...,...,...,...,...
179996,2019 its tough at first but fun when you get t...,197.3,2,Recommended,"December 15, 2019",Sekiro™: Shadows Die Twice - GOTY Edition
179997,2019 From the visuals to the exquisite game me...,91.5,2,Recommended,"December 16, 2019",Sekiro™: Shadows Die Twice - GOTY Edition
179998,2019 This game is really really cool. Beating ...,67.5,2,Recommended,"December 15, 2019",Sekiro™: Shadows Die Twice - GOTY Edition
179999,2019 yes,174.8,2,Recommended,"December 14, 2019",Sekiro™: Shadows Die Twice - GOTY Edition


In [6]:
df.to_excel("180001_game_review.xlsx", index=False)

Очистка от стоп слов, смайликов, токенизация, Леммитизация, Обработка сущностей

In [13]:
import re
import pandas as pd
from bs4 import BeautifulSoup

# === EMOJI UNICODE PATTERN ===
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FAFF"
    "\u2600-\u26FF"          # misc symbols
    "\u2700-\u27BF"
    "]+",
    flags=re.UNICODE
)

# === TEXT EMOJIS MAPPING ===
TEXT_EMOJI_MAP = {
    r":\)": "<EMOJI_POS>",
    r"=\)": "<EMOJI_POS>",
    r":d": "<EMOJI_POS>",
    r";\)": "<EMOJI_NEUTRAL>",
    r":\(": "<EMOJI_NEG>",
    r":-\(": "<EMOJI_NEG>",
    r":/": "<EMOJI_NEG>",
}

# === OPTIONAL: simple slang normalization ===
SLANG_MAP = {
    r"\bpls\b": "please",
    r"\bplz\b": "please",
    r"\basap\b": "as soon as possible",
    r"\burg\b": "urgent",
    r"\bu\b": "you",
    r"\br\b": "are",
    r"\btho\b": "though",
}

def preprocess_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # --- Remove HTML ---
    text = BeautifulSoup(text, "html.parser").get_text(" ")

    # --- Remove emojis ---
    text = EMOJI_PATTERN.sub(" ", text)


    # --- Replace emails ---
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', ' <EMAIL> ', text)

    # --- Replace URLs ---
    text = re.sub(r'https?://\S+|www\.\S+', ' <URL> ', text)

    # --- Replace IPs ---
    text = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', ' <IP> ', text)

    # --- Replace Ticket / IDs ---
    text = re.sub(r'\b[A-Z]{2,}-?\d{3,}\b', ' <ID> ', text)

    # --- Replace nicknames @username ---
    text = re.sub(r'@\w+', ' <USER> ', text)

    # --- Replace dates ---
    text = re.sub(r'\b\d{4}[-/]\d{2}[-/]\d{2}\b', ' <DATE> ', text)
    text = re.sub(r'\b\d{2}[-/]\d{2}[-/]\d{4}\b', ' <DATE> ', text)

    # --- Replace times ---
    text = re.sub(r'\b\d{1,2}:\d{2}\b', ' <TIME> ', text)

    # --- Replace text-based emojis ---
    for pattern, replacement in TEXT_EMOJI_MAP.items():
        text = re.sub(pattern, replacement, text)


    # --- Lowercase ---
    text = text.lower()

    # aaaaaaaa -> aaa
    text = re.sub(r"(.)\1{3,}", r"\1\1\1", text)

    # слова длиннее 30 символов -> удалить
    text = re.sub(r"\b\w{30,}\b", " ", text)


    # --- Slang normalization ---
    for pattern, replacement in SLANG_MAP.items():
        text = re.sub(pattern, replacement, text)

    # --- Remove repeated punctuation ---
    text = re.sub(r'([!?.,]){2,}', r'\1', text)

    # ____ -> _
    text = re.sub(r"([_\-])\1{2,}", r"\1", text)

    # !!!??? -> !
    text = re.sub(r"([!?]){2,}", r"\1", text)


    # --- Remove extra whitespace ---
    text = re.sub(r'\s+', ' ', text).strip()

    # --- Handle hashtags: remove # but keep word ---
    text = re.sub(r"#(\w+)", r"\1", text)

    return text

Удаление рисунков из точек и разных символов

In [14]:
import re
import unicodedata

# --- Регексы ---
UNICODE_ART_RANGES = (
    r"\u2800-\u28FF"  # Braille
    r"\u2500-\u257F"  # Box Drawing
    r"\u2580-\u259F"  # Block Elements
    r"\u25A0-\u25FF"  # Geometric Shapes
    r"\u2700-\u27BF"  # Dingbats
)

RE_UNICODE_ART = re.compile(f"[{UNICODE_ART_RANGES}]+")

RE_LONG_SYMBOL_SEQ = re.compile(
    r"[^\w\s\u0400-\u04FF\u4E00-\u9FFF]{5,}",  # 5+ небуквенных символов подряд
    flags=re.UNICODE
)

RE_MULTISPACE = re.compile(r"\s{2,}")


def clean_text_for_rag(text: str) -> str:
    if not text:
        return ""

    # 1. Удаляем unicode-art
    text = RE_UNICODE_ART.sub(" ", text)

    # 2. Удаляем длинные последовательности символов (ASCII-art)
    text = RE_LONG_SYMBOL_SEQ.sub(" ", text)

    # 3. Удаляем строки, где >40% символов — не буквы
    lines = []
    for line in text.splitlines():
        if not line.strip():
            continue

        letters = sum(c.isalpha() for c in line)
        ratio = letters / max(len(line), 1)

        if ratio > 0.4:
            lines.append(line)

    text = " ".join(lines)

    # 4. Нормализация пробелов
    text = RE_MULTISPACE.sub(" ", text)

    return text.strip()


In [15]:
df["review"] = df["review"].apply(preprocess_text)
df["review"] = df["review"].astype(str).apply(clean_text_for_rag)

In [16]:
df.to_excel('Clean_text.xlsx', index=False)